### **imports**

In [1]:
import pandas as pd
import numpy as np
from astropy.io import ascii
from astropy.table import QTable, Table, Column
from astropy import units as u
import string
import os
import upsilon

-------------------
| pyFFTW detected |
-------------------


### **making a table of all of the stars in the txt file**

In [2]:
column1 = []
column2 = []
column3 = []
column4 = []
column5 = []

with open('rrlyr_vsx_clean2.txt', 'r') as file:
    for line in file:
        components = line.split(' ', 4)
    
        try:
            magnitude = float(components[3])  # Try to convert the 4th component to float
        except ValueError:
            continue

        if magnitude < 15.5:
            column1.append(components[0])
            column2.append(float(components[1]))
            column3.append(float(components[2]))
            column4.append(magnitude)
            column5.append(components[4].strip())

final_table = QTable([column1, column2, column3, column4, column5], names = ("Star Type", "RA", "Dec", "Magnitude", "Other Info"))

In [3]:
final_table

Star Type,RA,Dec,Magnitude,Other Info
str24,float64,float64,float64,str29
RRAB,0.01521,35.36286,12.43,GM And
RRAB,0.01669,18.40698,15.171,CSS_J000004.0+182425
RRAB,0.03171,34.67397,14.95,SERIV 27
RRAB,0.06821,-72.77875,15.1,BP Tuc
RRAB,0.099,36.3287,14.46,NSVS 6313844
RRAB,0.1338,22.99388,14.671,CSS_J000032.1+225937
RRAB,0.13423,-54.29512,15.26,ASASSN-V J000032.21-541742.4
RRAB,0.14829,26.66375,12.96,GV Peg
RRC,0.15377,-60.98439,15.18,ASASSN-V J000036.92-605903.9


### **importing the asassn client**

In [4]:
from pyasassn.client import SkyPatrolClient	
client = SkyPatrolClient()

Welcome to ASAS-SN Skypatrol!

Current Deployment Version: 0.6.17 (26 JAN 2024)
Please upgrade your client if not up to date.



### **downloading a csv file with the data from the star at a specified index**

In [5]:
def classification_model(path, ra, dec, magnitude1, vsx_id, asassn_id):
  # Load a classification model.
  # (takes time, do only once if processing multiple lcs)
  rf_model = upsilon.load_rf_model()


  ############# Read input lc data #############
  # Read the input lightcurve file
  f = open(path, 'r')
  data = np.loadtxt(f, dtype={'names': ('JD', 'mag', 'err'),'formats': ('float64', 'float64', 'float64')})
  f.close()

  # Split it into three arrays
  date=data['JD']
  mag=data['mag']
  err=data['err']

  # Sigma-clip the input lightcurve
  date, mag, err = upsilon.utils.sigma_clipping(date, mag, err,  threshold=3, iteration=1)

  ############# Extract lightcurve features #############
  e_features = upsilon.ExtractFeatures(date, mag, err)
  e_features.run()
  features = e_features.get_features()

  ############# Classify #############
  # Classify the light curve
  label, probability, flag = upsilon.predict(rf_model, features)

  # Print results

  output_file = open(path.replace('.txt', '_classification.txt'), 'w')
  output_file.write("\n\n######### Classification results #########\n")
  output_file.write("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)\n")
  output_file.write("class = " + str(label) + "\n")
  output_file.write("class_probability = " + str(probability) + "\n")
  output_file.write("position = " + str(ra) + " " + str(dec) + "\n")
  output_file.write("magnitude = " + str(magnitude1) + "\n")
  output_file.write("vsx_id = " + str(vsx_id) + "\n")
  output_file.write("asassn_id = " + str(asassn_id) + "\n")
  output_file.close()

  print("\n\n######### Classification results #########\n")
  print("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)")
  print("class = " + str(label) )
  print("class_probability = " + str(probability) )
  print("\n\n######### Lightcurve features #########\n")

  for feature in features:
    print( str(feature) + " = " + str(features[feature]) )
  #  print feature," = ",features[feature]

In [8]:
folder_path = "/Users/zoesurles/Desktop/variable_star_classification/downloaded_lightcurves/"

In [19]:
num_iterations = 1
empty_list = []
for i in range(22, 32):
    if i == 8:
        continue
    search = client.cone_search(ra_deg = final_table["RA"][i], dec_deg = final_table["Dec"][i], radius=0.0003, catalog='aavsovsx', download=True)

    savecsv = search.save(save_dir = folder_path, file_format="csv")[1]
    empty_list.append(savecsv)

    new_file = pd.read_csv(savecsv, skiprows=1)

    g_filter = new_file[new_file['phot_filter'] == 'g']
    web_format = g_filter[["jd", "mag", "mag_err"]]
    web_format.to_csv(savecsv.replace(".csv", ".txt"), sep='\t', index=False, header = False)

    classification_model(savecsv.replace(".csv", ".txt"), final_table["RA"][i], final_table["Dec"][i], final_table["Magnitude"][i], final_table["Other Info"][i], new_file["asas_sn_id"][0])

Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 0.52


######### Lightcurve features #########

amplitude = 0.044237170336775
cusum = 0.07836319861786013
eta = 2.444897135266231
hl_amp_ratio = 0.9824825357520913
kurtosis = 0.15685558591066684
n_points = 1125
period = 0.6189024041584316
period_SNR = 90.26689328782167
period_log10FAP = -40.31232411443857
period_uncertainty = 0.00018383241349367863
phase_cusum = 0.14933564476638755
phase_eta = 1.7740874616948017
phi21 = 0.7407007433712769
phi31 = -1.132662781885561
quartile31 = 0.09509688378197723
r21 = 0.4912890440890094
r31 = 0.3297273022877241
shapiro_w = 0.9861631393432617
skewness = 0.2965290901586304
slope_per10 = -0.07277348814047863
slope_per90 = 0.04545132241334699
stetson_k = 0.787441826439107
weighted_mean = 13.363250281588574
weighted_std = 0.06772962007343057
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 0.75


######### Lightcurve features #########

amplitude = 0.5560048611055792
cusum = 0.4903841803262219
eta = 2.3772020152613482
hl_amp_ratio = 0.691126158089659
kurtosis = 0.6384544563381733
n_points = 1947
period = 0.4933605315059435
period_SNR = 228.15045488719417
period_log10FAP = -181.19853202583408
period_uncertainty = 0.00010523902601417001
phase_cusum = 0.6311650641499014
phase_eta = 1.4873780842545044
phi21 = -2.4185966497051554
phi31 = -4.3554291315741755
quartile31 = 0.7378111066986435
r21 = 0.4708786424887111
r31 = 0.31043107204744563
shapiro_w = 0.979256808757782
skewness = 0.4131652216688473
slope_per10 = -0.005592044103251481
slope_per90 = 0.005067611169766001
stetson_k = 0.7215095286149521
weighted_mean = 10.47179131283237
weighted_std = 0.44265130496398136
Pulled 1 of 1


...



######### Classification results #########

flag = 1 (0 - classification success, 1 - suspicious period)
class = EB_ED
class_probability = 0.78


######### Lightcurve features #########

amplitude = 0.01332647582235655
cusum = 0.10897766167483412
eta = 2.3385913948724752
hl_amp_ratio = 1.4419480732039065
kurtosis = 0.4319188701825447
n_points = 3707
period = 0.9972077959868334
period_SNR = 124.6156766043392
period_log10FAP = -56.874153755434776
period_uncertainty = 0.00043032565377681964
phase_cusum = 0.11667297723704446
phase_eta = 2.1380600320990566
phi21 = 0.5843675950543494
phi31 = -1.5025045773045969
quartile31 = 0.0370313419422601
r21 = 0.7625563503535636
r31 = 0.3184355973080921
shapiro_w = 0.9886456727981567
skewness = 0.2501332360076877
slope_per10 = -0.03331844901218557
slope_per90 = 0.03374931458492243
stetson_k = 0.6976320257241361
weighted_mean = 14.052525665125494
weighted_std = 0.028673311172009164
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 1 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 0.7


######### Lightcurve features #########

amplitude = 0.2993896294414602
cusum = 0.24714801098029376
eta = 2.0805833570520997
hl_amp_ratio = 0.8339206619007343
kurtosis = 0.5707926453197438
n_points = 2225
period = 0.5189515141469936
period_SNR = 224.62188728385286
period_log10FAP = -174.65672821928507
period_uncertainty = 9.689885942143484e-05
phase_cusum = 0.33966987124027914
phase_eta = 1.39380580439873
phi21 = 0.5640805828017006
phi31 = 1.3848058367458054
quartile31 = 0.4113537397297282
r21 = 0.3116788143224389
r31 = 0.12028751918574032
shapiro_w = 0.9658369421958923
skewness = 0.487332109500544
slope_per10 = -0.008950762810614172
slope_per90 = 0.00975602522748133
stetson_k = 0.7338204896603262
weighted_mean = 14.789400516479953
weighted_std = 0.3124768344904224
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.30437632379418883
cusum = 0.20150841336179579
eta = 2.0391028532031394
hl_amp_ratio = 0.47361656403740743
kurtosis = -0.7754861035531273
n_points = 1298
period = 0.5625743614562824
period_SNR = 177.78793619987349
period_log10FAP = -159.2503060800636
period_uncertainty = 0.0001137801115503323
phase_cusum = 0.3644881497500779
phase_eta = 0.5123122179853062
phi21 = 0.742329787106943
phi31 = 1.9099229207868051
quartile31 = 0.44099120177449436
r21 = 0.47048617496249534
r31 = 0.23021293919481092
shapiro_w = 0.9551767110824585
skewness = -0.44778230492160837
slope_per10 = -0.022244258911947444
slope_per90 = 0.024686236678494674
stetson_k = 0.7037549799554151
weighted_mean = 15.410262303016603
weighted_std = 0.27497991816924616
